**Práctica: Eliminación Contextual de Stopwords para Análisis de Reseñas de Productos**  

**Contexto:** Eres un NLP Engineer en **ReviewBoost**, una startup que analiza reseñas de Amazon para identificar problemas críticos en productos.  

---

### **Fase 1: Análisis Inicial**  
**Objetivo:** Entender cómo las stopwords por defecto afectan el significado en reseñas.  

#### **Tarea 1:**  
- **Descargar** el dataset `comensarios_clintes.csv`. Puedes utilizar otro dataset de internet.  
- **Ejemplo de Entrada:**  
```No recomiendo este producto. Aunque es barato, se rompió en dos días.
``` 
- **Procesar** el texto usando spaCy con stopwords por defecto.  
- **Registrar:**  
  - ¿Qué palabras clave se eliminaron (ej. "no", "aunque")?  
  - ¿Cómo afecta esto al significado?  

**Procesamiento Actual (spaCy):**  
```  
["mal", "producto", "entrega", "pésima", "marketminddecepciona"]  
```  
**Problema:** La palabra "no" se eliminó, invirtiendo el significado.  

#### **Pistas:**  
- Usar `spacy.load("es_core_news_sm")` y `token.is_stop`.  
- Para debuggear: Imprimir lista de stopwords con `print(nlp.Defaults.stop_words)`.  

#### **Verificación:**  
El estudiante debe generar una tabla con 5 ejemplos donde la eliminación de stopwords alteró el significado.  
 

---

### **Fase 2: Personalización de la Lista**  
**Objetivo:** Crear una lista de stopwords adaptada a reseñas de productos.  

#### **Tarea 2:**  
1. **Preservar Términos Clave:**  
   - Negaciones: "no", "nunca", "tampoco".  
   - Conectores de contraste: "pero", "aunque", "sin embargo".  
2. **Eliminar Términos Genéricos:**  
   - Palabras redundantes: "producto", "cliente", "día".  
   - Verbos comunes sin contexto: "hacer", "tener", "decir".  
3. **Añadir Stopwords Específicas:**  
   - Términos no informativos: "hola", "gracias", "pd".  

#### **Pasos:**  
- **Analizar Frecuencia:** Usar `Counter` de Python para identificar palabras repetidas en el 90% de las reseñas.  
- **Modificar Lista:**  
  - Cargar stopwords de spaCy.  
  - Quitar términos críticos (ej. `stopwords_es.discard("no")`).  
  - Añadir términos redundantes (ej. `stopwords_es.add("producto")`).  

#### **Pistas:**  
- Para "hola" y "gracias", usar regex: `r'\b(hola|gracias)\b'`.  
- Usar `nltk.corpus.stopwords.words('spanish')` como lista alternativa si hay inconsistencias.  

---

### **Fase 3: Implementación y Pruebas**  
**Objetivo:** Validar que la lista personalizada preserva el contexto crítico.  

#### **Tarea 3:**  
1. **Función de Procesamiento:**  
   - Input: Texto crudo.  
   - Output: Lista de tokens sin stopwords personalizadas.  
2. **Casos de Prueba:**  
   ```  
   Texto 1: "No funciona bien, pero el diseño es bonito."  
   Output Esperado: ["no", "funciona", "bien", "pero", "diseño", "bonito"]  

   Texto 2: "Nunca compré algo tan malo. Aunque el precio es bajo, no lo vale."  
   Output Esperado: ["nunca", "compré", "malo", "aunque", "precio", "bajo", "no", "vale"]  
   ```  
3. **Métricas:**  
   - Precisión: 100% de los términos clave preservados en 20 casos de prueba predefinidos.  

#### **Pistas:**  
- Usar `token.text.lower()` para normalizar.  
- Si "aunque" se elimina, revisar `stopwords_es.remove("aunque")`.  

---

### **Fase 4: Evaluación de Impacto**  
**Objetivo:** Medir cómo afecta la personalización al análisis de sentimiento.  

#### **Tarea 4:**  
1. **Entrenar Modelo Básico:**  
   - Usar `sklearn` (CountVectorizer + LogisticRegression).  
   - Comparar resultados con/sin stopwords personalizadas.  
2. **Métricas:**  
   - Exactitud (accuracy) en un subset de 200 reseñas etiquetadas manualmente.  
3. **Análisis:**  
   - ¿En qué reseñas mejoró/deterioró la clasificación?  

#### **Pistas:**  
- Usar `max_features=1000` en CountVectorizer para reducir dimensionalidad.  
- Ejemplo de mejora: Reseñas con negaciones clasificadas correctamente.  

---

### **Entrega Final**  
1. **Código:**  
   - Script `stopwords_custom.py` (funciones de carga, procesamiento, y evaluación).  
2. **Documentación:**  
   - Lista final de stopwords (`.txt`).  



In [1]:
# Importación de librerías necesarias
import pandas as pd
import spacy
import re
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Cargar el modelo de spaCy en español
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    print("Descargando modelo de spaCy para español...")
    !python -m spacy download es_core_news_sm
    nlp = spacy.load("es_core_news_sm")

# Configuración para visualizaciones
plt.style.use('ggplot')
sns.set(style='whitegrid')


In [2]:
# Cargar el dataset de comentarios
try:
    df = pd.read_csv('comentarios_clientes.csv')
    print(f"Dataset cargado correctamente. Shape: {df.shape}")
    print("\nPrimeras 5 filas:")
    display(df.head())
    
    print("\nDistribución de etiquetas:")
    display(df['etiqueta'].value_counts())
except Exception as e:
    print(f"Error al cargar el dataset: {e}")


Dataset cargado correctamente. Shape: (50, 2)

Primeras 5 filas:


,texto,etiqueta
0,"El paquete llegó con el embalaje comprometido,...",neutral
1,"Después de 3 semanas de espera, el artículo nu...",negativo
2,La atención al cliente fue evasiva y poco reso...,negativo
3,Entrega express y producto en perfecto estado....,positivo
4,El sistema de seguimiento en línea mostró info...,negativo



Distribución de etiquetas:


etiqueta
negativo    25
positivo    19
neutral      6
Name: count, dtype: int64

## Implementación Fase 1: Análisis Inicial
A continuación, analizaremos cómo las stopwords por defecto afectan el significado de las reseñas.

In [3]:
# Fase 1: Función para procesar texto con stopwords por defecto de spaCy
def procesar_texto_default(texto):
    """Procesa texto usando spaCy con las stopwords por defecto."""
    doc = nlp(texto)
    # Extraer tokens que no son stopwords y no son signos de puntuación
    tokens = [token.text.lower() for token in doc if not token.is_stop and not token.is_punct]
    return tokens

# Función para procesar texto pero conservando stopwords (para comparación)
def procesar_texto_con_stopwords(texto):
    """Procesa texto usando spaCy pero conservando stopwords."""
    doc = nlp(texto)
    # Extraer tokens que no son signos de puntuación
    tokens = [token.text.lower() for token in doc if not token.is_punct]
    return tokens

# Ver la lista de stopwords en español de spaCy
print("Total de stopwords en spaCy (español):", len(nlp.Defaults.stop_words))
print("Muestra de 15 stopwords:", list(nlp.Defaults.stop_words)[:15])

# Analizar ejemplos para ver el impacto de eliminar stopwords
ejemplos = [
    "No recomiendo este producto. Aunque es barato, se rompió en dos días.",
    "El producto nunca funcionó correctamente, pero el servicio al cliente fue excelente.",
    "Aunque el embalaje estaba dañado, el producto llegó en buenas condiciones.",
    "No me gustó la calidad, sin embargo el precio es muy bueno.",
    "Nunca había visto algo tan malo como este artículo."  
]

# Crear tabla comparativa
resultados = []
for i, ejemplo in enumerate(ejemplos, 1):
    con_sw = procesar_texto_con_stopwords(ejemplo)
    sin_sw = procesar_texto_default(ejemplo)
    
    # Identificar palabras clave eliminadas
    eliminadas = [w for w in con_sw if w not in sin_sw and w in nlp.Defaults.stop_words]
    
    resultados.append({
        "Ejemplo": i,
        "Texto Original": ejemplo,
        "Con Stopwords": con_sw,
        "Sin Stopwords (Default)": sin_sw,
        "Palabras Clave Eliminadas": eliminadas,
        "Impacto en Significado": "Analizar manualmente"
    })

# Mostrar tabla de resultados
df_resultados = pd.DataFrame(resultados)
display(df_resultados[['Ejemplo', 'Texto Original', 'Sin Stopwords (Default)', 'Palabras Clave Eliminadas']])

# Información para análisis manual
print("\nAnálisis manual del impacto en el significado:")
for i, ejemplo in enumerate(ejemplos, 1):
    print(f"\nEjemplo {i}: {ejemplo}")
    print(f"Palabras clave eliminadas: {resultados[i-1]['Palabras Clave Eliminadas']}")
    print("Describir cómo esto afecta el significado: [COMPLETAR MANUALMENTE]")


Total de stopwords en spaCy (español): 521
Muestra de 15 stopwords: ['menos', 'tu', 'ahí', 'nuevo', 'indicó', 'vosotras', 'último', 'cualquier', 'según', 'muchas', 'ninguno', 'se', 'esta', 'demasiado', 'lado']


,Ejemplo,Texto Original,Sin Stopwords (Default),Palabras Clave Eliminadas
0,1,"No recomiendo este producto. Aunque es barato,...","[recomiendo, producto, barato, rompió]","[no, este, aunque, es, se, en, dos, días]"
1,2,"El producto nunca funcionó correctamente, pero...","[producto, funcionó, correctamente, servicio, ...","[el, nunca, pero, el, al, fue]"
2,3,"Aunque el embalaje estaba dañado, el producto ...","[embalaje, dañado, producto, condiciones]","[aunque, el, estaba, el, llegó, en, buenas]"
3,4,"No me gustó la calidad, sin embargo el precio ...","[gustó, calidad, precio]","[no, me, la, sin, embargo, el, es, muy, bueno]"
4,5,Nunca había visto algo tan malo como este artí...,"[visto, malo, artículo]","[nunca, había, algo, tan, como, este]"



Análisis manual del impacto en el significado:

Ejemplo 1: No recomiendo este producto. Aunque es barato, se rompió en dos días.
Palabras clave eliminadas: ['no', 'este', 'aunque', 'es', 'se', 'en', 'dos', 'días']
Describir cómo esto afecta el significado: [COMPLETAR MANUALMENTE]

Ejemplo 2: El producto nunca funcionó correctamente, pero el servicio al cliente fue excelente.
Palabras clave eliminadas: ['el', 'nunca', 'pero', 'el', 'al', 'fue']
Describir cómo esto afecta el significado: [COMPLETAR MANUALMENTE]

Ejemplo 3: Aunque el embalaje estaba dañado, el producto llegó en buenas condiciones.
Palabras clave eliminadas: ['aunque', 'el', 'estaba', 'el', 'llegó', 'en', 'buenas']
Describir cómo esto afecta el significado: [COMPLETAR MANUALMENTE]

Ejemplo 4: No me gustó la calidad, sin embargo el precio es muy bueno.
Palabras clave eliminadas: ['no', 'me', 'la', 'sin', 'embargo', 'el', 'es', 'muy', 'bueno']
Describir cómo esto afecta el significado: [COMPLETAR MANUALMENTE]

Ejemplo 5: Nu

## Implementación Fase 2: Personalización de la Lista de Stopwords
A continuación, crearemos una lista personalizada de stopwords adaptada al análisis de reseñas de productos.

In [4]:
# Fase 2: Personalización de stopwords

# 1. Analizar frecuencia de palabras en el dataset
def analizar_frecuencia(df, columna_texto):
    """Analiza la frecuencia de palabras en un dataset."""
    # Concatenar todos los textos
    texto_completo = ' '.join(df[columna_texto].astype(str).values)
    # Procesar con spaCy
    doc = nlp(texto_completo)
    # Crear contador de palabras (excluyendo puntuación)
    palabras = [token.text.lower() for token in doc if not token.is_punct]
    contador = Counter(palabras)
    return contador

# Obtener frecuencias
frequencias = analizar_frecuencia(df, 'texto')
print("Palabras más comunes en el dataset:")
print(frequencias.most_common(20))

# 2. Personalizar lista de stopwords

# Obtener lista base de stopwords
stopwords_personalizadas = set(nlp.Defaults.stop_words)

# Términos clave a preservar (eliminar de la lista de stopwords)
terminos_a_preservar = [
    "no", "nunca", "tampoco",  # Negaciones
    "pero", "aunque", "sin", "embargo"  # Conectores de contraste
]

# Términos genéricos a eliminar (añadir a la lista de stopwords)
terminos_genericos = [
    "producto", "cliente", "día",  # Palabras redundantes
    "hacer", "tener", "decir",     # Verbos comunes sin contexto
    "hola", "gracias", "pd"       # Términos no informativos
]

# Modificar la lista de stopwords
for termino in terminos_a_preservar:
    if termino in stopwords_personalizadas:
        stopwords_personalizadas.remove(termino)
        print(f"Término preservado: '{termino}'")

for termino in terminos_genericos:
    stopwords_personalizadas.add(termino)
    print(f"Término añadido: '{termino}'")

# 3. Palabras muy frecuentes en el dataset que podrían ser stopwords
palabras_frecuentes = [palabra for palabra, freq in frequencias.most_common(30) 
                      if len(palabra) > 2 and palabra not in terminos_a_preservar]

print("\nPalabras frecuentes que podrían añadirse como stopwords:")
print(palabras_frecuentes[:15])

# Añadir palabras muy frecuentes (con cuidado)
for palabra in palabras_frecuentes[:10]:
    if palabra not in terminos_a_preservar and len(palabra) > 2:
        stopwords_personalizadas.add(palabra)
        print(f"Añadida palabra frecuente: '{palabra}'")

# Mostrar tamaño de las listas de stopwords
print(f"\nTotal stopwords originales: {len(nlp.Defaults.stop_words)}")
print(f"Total stopwords personalizadas: {len(stopwords_personalizadas)}")

# Guardar la lista personalizada de stopwords
with open('stopwords_personalizadas.txt', 'w', encoding='utf-8') as f:
    for word in sorted(stopwords_personalizadas):
        f.write(word + '\n')

print("Lista de stopwords personalizadas guardada en 'stopwords_personalizadas.txt'")


Palabras más comunes en el dataset:
[('de', 39), ('en', 22), ('con', 14), ('el', 10), ('del', 8), ('la', 7), ('para', 6), ('a', 5), ('productos', 5), ('artículo', 4), ('y', 4), ('seguimiento', 3), ('mi', 3), ('producto', 3), ('estado', 3), ('sistema', 3), ('proceso', 3), ('pero', 3), ('que', 3), ('pago', 3)]
Término preservado: 'no'
Término preservado: 'nunca'
Término preservado: 'tampoco'
Término preservado: 'pero'
Término preservado: 'aunque'
Término preservado: 'sin'
Término preservado: 'embargo'
Término añadido: 'producto'
Término añadido: 'cliente'
Término añadido: 'día'
Término añadido: 'hacer'
Término añadido: 'tener'
Término añadido: 'decir'
Término añadido: 'hola'
Término añadido: 'gracias'
Término añadido: 'pd'

Palabras frecuentes que podrían añadirse como stopwords:
['con', 'del', 'para', 'productos', 'artículo', 'seguimiento', 'producto', 'estado', 'sistema', 'proceso', 'que', 'pago', 'programa', 'opciones', 'problemas']
Añadida palabra frecuente: 'con'
Añadida palabra fre

## Implementación Fase 3: Implementación y Pruebas
A continuación, implementaremos la función de procesamiento utilizando nuestra lista personalizada de stopwords y la probaremos con casos específicos.

In [5]:
# Fase 3: Implementación y pruebas

# 1. Función de procesamiento con stopwords personalizadas
def procesar_texto_personalizado(texto, stopwords_personalizadas):
    """Procesa texto eliminando stopwords personalizadas.
    
    Args:
        texto (str): Texto a procesar
        stopwords_personalizadas (set): Conjunto de stopwords personalizadas
        
    Returns:
        list: Lista de tokens sin stopwords personalizadas
    """
    doc = nlp(texto)
    # Extraer tokens que no son stopwords personalizadas y no son signos de puntuación
    tokens = [token.text.lower() for token in doc 
              if token.text.lower() not in stopwords_personalizadas 
              and not token.is_punct]
    return tokens

# 2. Casos de prueba
casos_prueba = [
    {
        "texto": "No funciona bien, pero el diseño es bonito.",
        "esperado": ["no", "funciona", "bien", "pero", "diseño", "bonito"]
    },
    {
        "texto": "Nunca compré algo tan malo. Aunque el precio es bajo, no lo vale.",
        "esperado": ["nunca", "compré", "malo", "aunque", "precio", "bajo", "no", "vale"]
    },
    {
        "texto": "El paquete llegó con el embalaje comprometido, aunque lograron reembolsarme rápidamente.",
        "esperado": ["paquete", "llegó", "embalaje", "comprometido", "aunque", "lograron", "reembolsarme", "rápidamente"]
    },
    {
        "texto": "No me gustó para nada la calidad del material, pero el precio es justo.",
        "esperado": ["no", "gustó", "nada", "calidad", "material", "pero", "precio", "justo"]
    },
    {
        "texto": "Nunca había visto un servicio tan malo como este.",
        "esperado": ["nunca", "visto", "servicio", "tan", "malo"]
    }
]

# 3. Evaluación de los casos de prueba
resultados_prueba = []
for i, caso in enumerate(casos_prueba, 1):
    resultado = procesar_texto_personalizado(caso["texto"], stopwords_personalizadas)
    
    # Comparar con el resultado esperado
    terminos_faltantes = [t for t in caso["esperado"] if t not in resultado]
    terminos_extra = [t for t in resultado if t not in caso["esperado"]]
    coincide = (terminos_faltantes == [] and terminos_extra == [])
    precision = len(set(caso["esperado"]) & set(resultado)) / len(set(caso["esperado"])) * 100
    
    resultados_prueba.append({
        "Caso": i,
        "Texto": caso["texto"],
        "Resultado": resultado,
        "Esperado": caso["esperado"],
        "Términos faltantes": terminos_faltantes,
        "Términos extra": terminos_extra,
        "¿Coincide?": coincide,
        "Precisión (%)": precision
    })
    
# Mostrar resultados
df_pruebas = pd.DataFrame(resultados_prueba)
display(df_pruebas[['Caso', 'Texto', 'Resultado', 'Esperado', '¿Coincide?', 'Precisión (%)']])

# Resumen de precisión
precision_total = sum(df_pruebas['Precisión (%)']) / len(df_pruebas)
print(f"\nPrecisión promedio: {precision_total:.2f}%")

# Ajuste fino: Si hay términos problemáticos, ajustar la lista
problemas = df_pruebas[df_pruebas['Precisión (%)'] < 100]
if not problemas.empty:
    print("\nCasos con problemas que requieren ajuste:")
    display(problemas[['Caso', 'Texto', 'Términos faltantes', 'Términos extra']])
    
    # Ajustar la lista de stopwords si es necesario
    print("\nAjustando lista de stopwords...")
    for _, caso in problemas.iterrows():
        for termino in caso['Términos faltantes']:
            if termino in stopwords_personalizadas:
                stopwords_personalizadas.remove(termino)
                print(f"Quitando '{termino}' de las stopwords")


,Caso,Texto,Resultado,Esperado,¿Coincide?,Precisión (%)
0,1,"No funciona bien, pero el diseño es bonito.","[no, funciona, pero, diseño, bonito]","[no, funciona, bien, pero, diseño, bonito]",False,83.333333
1,2,Nunca compré algo tan malo. Aunque el precio e...,"[nunca, compré, malo, aunque, precio, no, vale]","[nunca, compré, malo, aunque, precio, bajo, no...",False,87.500000
2,3,"El paquete llegó con el embalaje comprometido,...","[paquete, embalaje, comprometido, aunque, logr...","[paquete, llegó, embalaje, comprometido, aunqu...",False,87.500000
3,4,"No me gustó para nada la calidad del material,...","[no, gustó, calidad, material, pero, precio, j...","[no, gustó, nada, calidad, material, pero, pre...",False,87.500000
4,5,Nunca había visto un servicio tan malo como este.,"[nunca, visto, servicio, malo]","[nunca, visto, servicio, tan, malo]",False,80.000000



Precisión promedio: 85.17%

Casos con problemas que requieren ajuste:


,Caso,Texto,Términos faltantes,Términos extra
0,1,"No funciona bien, pero el diseño es bonito.",[bien],[]
1,2,Nunca compré algo tan malo. Aunque el precio e...,[bajo],[]
2,3,"El paquete llegó con el embalaje comprometido,...",[llegó],[]
3,4,"No me gustó para nada la calidad del material,...",[nada],[]
4,5,Nunca había visto un servicio tan malo como este.,[tan],[]



Ajustando lista de stopwords...
Quitando 'bien' de las stopwords
Quitando 'bajo' de las stopwords
Quitando 'llegó' de las stopwords
Quitando 'nada' de las stopwords
Quitando 'tan' de las stopwords


## Implementación Fase 4: Evaluación de Impacto
A continuación, evaluaremos el impacto de nuestra lista personalizada de stopwords en el análisis de sentimiento de las reseñas.

In [6]:
# Fase 4: Evaluación de Impacto en Análisis de Sentimiento

# Preparar los datos para entrenamiento
# Nos aseguramos que el dataset tiene al menos las columnas 'texto' y 'etiqueta'
assert 'texto' in df.columns and 'etiqueta' in df.columns, "El dataset debe tener columnas 'texto' y 'etiqueta'"

# Preparar función para preprocesamiento
def preprocesar_dataset(df, columna_texto, usar_stopwords_personalizadas=False):
    """Preprocesa los textos del dataset.
    
    Args:
        df (DataFrame): DataFrame con los textos
        columna_texto (str): Nombre de la columna con los textos
        usar_stopwords_personalizadas (bool): Si True, usa las stopwords personalizadas
        
    Returns:
        list: Lista de textos procesados
    """
    textos_procesados = []
    for texto in df[columna_texto]:
        if usar_stopwords_personalizadas:
            tokens = procesar_texto_personalizado(texto, stopwords_personalizadas)
        else:
            tokens = procesar_texto_default(texto)
        textos_procesados.append(' '.join(tokens))
    return textos_procesados

# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    df['texto'], df['etiqueta'], test_size=0.3, random_state=42, stratify=df['etiqueta'])

# Preprocesar los textos
# 1. Usando stopwords por defecto
X_train_default = preprocesar_dataset(pd.DataFrame({'texto': X_train}), 'texto', False)
X_test_default = preprocesar_dataset(pd.DataFrame({'texto': X_test}), 'texto', False)

# 2. Usando stopwords personalizadas
X_train_custom = preprocesar_dataset(pd.DataFrame({'texto': X_train}), 'texto', True)
X_test_custom = preprocesar_dataset(pd.DataFrame({'texto': X_test}), 'texto', True)

# Vectorizar los textos
vectorizer_default = CountVectorizer(max_features=1000)
vectorizer_custom = CountVectorizer(max_features=1000)

# Vectorización con stopwords por defecto
X_train_default_vec = vectorizer_default.fit_transform(X_train_default)
X_test_default_vec = vectorizer_default.transform(X_test_default)

# Vectorización con stopwords personalizadas
X_train_custom_vec = vectorizer_custom.fit_transform(X_train_custom)
X_test_custom_vec = vectorizer_custom.transform(X_test_custom)

# Entrenar modelo con stopwords por defecto
model_default = LogisticRegression(max_iter=1000, random_state=42)
model_default.fit(X_train_default_vec, y_train)

# Entrenar modelo con stopwords personalizadas
model_custom = LogisticRegression(max_iter=1000, random_state=42)
model_custom.fit(X_train_custom_vec, y_train)

# Evaluar modelos
y_pred_default = model_default.predict(X_test_default_vec)
y_pred_custom = model_custom.predict(X_test_custom_vec)

# Calcular métricas
accuracy_default = accuracy_score(y_test, y_pred_default)
accuracy_custom = accuracy_score(y_test, y_pred_custom)

print(f"Exactitud con stopwords por defecto: {accuracy_default:.4f}")
print(f"Exactitud con stopwords personalizadas: {accuracy_custom:.4f}")
print(f"Mejora: {(accuracy_custom - accuracy_default)*100:.2f}%")

# Reportes detallados
print("\nClasificación con stopwords por defecto:")
print(classification_report(y_test, y_pred_default))

print("\nClasificación con stopwords personalizadas:")
print(classification_report(y_test, y_pred_custom))

# Análisis de ejemplos donde difieren las predicciones
resultados_comparativos = []
for i, (texto, etiqueta_real) in enumerate(zip(X_test, y_test)):
    pred_default = y_pred_default[i]
    pred_custom = y_pred_custom[i]
    
    if pred_default != pred_custom:
        resultados_comparativos.append({
            "Texto": texto,
            "Etiqueta Real": etiqueta_real,
            "Predicción Default": pred_default,
            "Predicción Custom": pred_custom,
            "Custom Correcta": pred_custom == etiqueta_real,
            "Default Correcta": pred_default == etiqueta_real
        })

# Mostrar ejemplos donde difieren las predicciones
if resultados_comparativos:
    df_comparativa = pd.DataFrame(resultados_comparativos)
    print(f"\nEncontrados {len(df_comparativa)} ejemplos donde las predicciones difieren:")
    display(df_comparativa)
    
    # Contar cuántas veces cada modelo acertó en estos casos
    mejora_custom = df_comparativa["Custom Correcta"].sum()
    mejora_default = df_comparativa["Default Correcta"].sum()
    print(f"El modelo con stopwords personalizadas acertó en {mejora_custom} de estos casos.")
    print(f"El modelo con stopwords por defecto acertó en {mejora_default} de estos casos.")
else:
    print("\nNo se encontraron diferencias en las predicciones de ambos modelos.")


Exactitud con stopwords por defecto: 0.6000
Exactitud con stopwords personalizadas: 0.4667
Mejora: -13.33%

Clasificación con stopwords por defecto:
              precision    recall  f1-score   support

    negativo       0.54      1.00      0.70         7
     neutral       0.00      0.00      0.00         2
    positivo       1.00      0.33      0.50         6

    accuracy                           0.60        15
   macro avg       0.51      0.44      0.40        15
weighted avg       0.65      0.60      0.53        15


Clasificación con stopwords personalizadas:
              precision    recall  f1-score   support

    negativo       0.47      1.00      0.64         7
     neutral       0.00      0.00      0.00         2
    positivo       0.00      0.00      0.00         6

    accuracy                           0.47        15
   macro avg       0.16      0.33      0.21        15
weighted avg       0.22      0.47      0.30        15


Encontrados 2 ejemplos donde las prediccion

/mnt/c/Users/aleja/Desktop/Master/05 - IA Generativa/Desarrollo/13-NLP/LimpiezaInicialTexto/.venvWSL/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/c/Users/aleja/Desktop/Master/05 - IA Generativa/Desarrollo/13-NLP/LimpiezaInicialTexto/.venvWSL/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/mnt/c/Users/aleja/Desktop/Master/05 - IA Generativa/Desarrollo/13-NLP/LimpiezaInicialTexto/.venvWSL/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetric

,Texto,Etiqueta Real,Predicción Default,Predicción Custom,Custom Correcta,Default Correcta
0,Experiencia realidad aumentada para visualizar...,positivo,positivo,negativo,False,True
1,Programa de pruebas gratuitas para productos d...,positivo,positivo,negativo,False,True


El modelo con stopwords personalizadas acertó en 0 de estos casos.
El modelo con stopwords por defecto acertó en 2 de estos casos.


## Implementación Final: Creación del script stopwords_custom.py
A continuación, crearemos el script `stopwords_custom.py` que contendrá las funciones desarrolladas.

In [7]:
# Crear el script stopwords_custom.py

# Definir contenido del script
script_content = '''
"""Módulo para la gestión personalizada de stopwords en análisis de reseñas.

Este módulo contiene funciones para cargar, personalizar y aplicar stopwords
en el contexto de análisis de reseñas de productos, preservando términos
críticos como negaciones y conectores de contraste.
"""

import spacy
import pandas as pd
from collections import Counter

# Cargar modelo de spaCy
try:
    nlp = spacy.load("es_core_news_sm")
except OSError:
    print("El modelo de spaCy para español no está instalado.")
    print("Instálalo con: python -m spacy download es_core_news_sm")
    raise


def cargar_stopwords_personalizadas(ruta_archivo=None):
    """Carga la lista personalizada de stopwords.
    
    Args:
        ruta_archivo (str, optional): Ruta al archivo con stopwords personalizadas.
            Si es None, retorna la lista predefinida en este módulo.
    
    Returns:
        set: Conjunto de stopwords personalizadas
    """
    if ruta_archivo:
        try:
            with open(ruta_archivo, "r", encoding="utf-8") as f:
                return set(line.strip() for line in f)
        except FileNotFoundError:
            print(f"Archivo {ruta_archivo} no encontrado. Usando lista predefinida.")
    
    # Lista predefinida (stopwords de spaCy con modificaciones)
    stopwords_personalizadas = set(nlp.Defaults.stop_words)
    
    # Términos clave a preservar
    terminos_a_preservar = [
        "no", "nunca", "tampoco",  # Negaciones
        "pero", "aunque", "sin", "embargo"  # Conectores de contraste
    ]
    
    # Términos genéricos a eliminar
    terminos_genericos = [
        "producto", "cliente", "día",  # Palabras redundantes
        "hacer", "tener", "decir",     # Verbos comunes sin contexto
        "hola", "gracias", "pd"       # Términos no informativos
    ]
    
    # Modificar la lista
    for termino in terminos_a_preservar:
        if termino in stopwords_personalizadas:
            stopwords_personalizadas.remove(termino)
            
    for termino in terminos_genericos:
        stopwords_personalizadas.add(termino)
        
    return stopwords_personalizadas


def procesar_texto(texto, stopwords=None, conservar_stopwords=False):
    """Procesa texto eliminando o conservando stopwords.
    
    Args:
        texto (str): Texto a procesar
        stopwords (set, optional): Conjunto de stopwords a utilizar.
            Si es None, usa las stopwords por defecto de spaCy.
        conservar_stopwords (bool): Si es True, conserva todas las stopwords.
    
    Returns:
        list: Lista de tokens procesados
    """
    doc = nlp(texto)
    
    if conservar_stopwords:
        return [token.text.lower() for token in doc if not token.is_punct]
    
    if stopwords is None:
        # Usar stopwords por defecto de spaCy
        return [token.text.lower() for token in doc 
                if not token.is_stop and not token.is_punct]
    else:
        # Usar lista personalizada de stopwords
        return [token.text.lower() for token in doc 
                if token.text.lower() not in stopwords and not token.is_punct]


def analizar_frecuencia(textos):
    """Analiza la frecuencia de palabras en un conjunto de textos.
    
    Args:
        textos (list or Series): Lista o Series de textos a analizar
    
    Returns:
        Counter: Contador con las frecuencias de palabras
    """
    if isinstance(textos, pd.Series):
        textos = textos.astype(str).values
        
    # Concatenar todos los textos
    texto_completo = " ".join(textos)
    doc = nlp(texto_completo)
    
    # Contar palabras (excluyendo puntuación)
    palabras = [token.text.lower() for token in doc if not token.is_punct]
    return Counter(palabras)


def guardar_stopwords(stopwords, ruta_archivo):
    """Guarda la lista de stopwords en un archivo.
    
    Args:
        stopwords (set): Conjunto de stopwords a guardar
        ruta_archivo (str): Ruta donde guardar el archivo
    """
    with open(ruta_archivo, "w", encoding="utf-8") as f:
        for word in sorted(stopwords):
            f.write(word + "\n")
    print(f"Lista de stopwords guardada en {ruta_archivo}")


def evaluar_precision(casos_prueba, stopwords):
    """Evalúa la precisión de la lista de stopwords en casos de prueba.
    
    Args:
        casos_prueba (list): Lista de diccionarios con casos de prueba.
            Cada diccionario debe tener las claves "texto" y "esperado".
        stopwords (set): Conjunto de stopwords a evaluar
    
    Returns:
        float: Precisión promedio (0-100)
    """
    resultados = []
    for caso in casos_prueba:
        resultado = procesar_texto(caso["texto"], stopwords)
        precision = len(set(caso["esperado"]) & set(resultado)) / len(set(caso["esperado"])) * 100
        resultados.append(precision)
    
    return sum(resultados) / len(resultados)


if __name__ == "__main__":
    # Ejemplo de uso
    print("Módulo de procesamiento de stopwords personalizadas")
    print("Ejemplo de uso:")
    
    texto_ejemplo = "No recomiendo este producto. Aunque es barato, se rompió en dos días."
    
    # Procesamiento con stopwords por defecto
    print("\nProcesamiento con stopwords por defecto:")
    tokens_default = procesar_texto(texto_ejemplo)
    print(tokens_default)
    
    # Procesamiento con stopwords personalizadas
    print("\nProcesamiento con stopwords personalizadas:")
    stopwords_custom = cargar_stopwords_personalizadas()
    tokens_custom = procesar_texto(texto_ejemplo, stopwords_custom)
    print(tokens_custom)
'''

# Guardar el script
with open('stopwords_custom.py', 'w', encoding='utf-8') as f:
    f.write(script_content)

print("Script stopwords_custom.py creado exitosamente.")

# Verificar que se puede importar
try:
    from stopwords_custom import cargar_stopwords_personalizadas, procesar_texto
    print("El módulo se importó correctamente.")
    
    # Probar el módulo
    sw = cargar_stopwords_personalizadas()
    ejemplo = "No recomiendo este producto. Aunque es barato, se rompió en dos días."
    resultado = procesar_texto(ejemplo, sw)
    print(f"\nProcesamiento con módulo importado:\n{resultado}")
except Exception as e:
    print(f"Error al importar el módulo: {e}")


Script stopwords_custom.py creado exitosamente.
Error al importar el módulo: unterminated string literal (detected at line 123) (stopwords_custom.py, line 123)
